In [1]:
%pwd

'e:\\Rag-gen-ai-project\\shopping-chatbot\\research'

In [2]:
import os 
os.chdir("../")

In [3]:
from langchain.document_loaders import UnstructuredURLLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [4]:
import requests
from bs4 import BeautifulSoup
import time

headers = {"User-Agent": "Mozilla/5.0"}
categories = ["laptops", "mobiles", "headphones"]
product_links = []

for category in categories:
    print(f"Fetching category: {category}")
    for page in range(1, 6):  # scrape first 5 pages per category
        url = f"https://www.flipkart.com/search?q={category}&page={page}"
        response = requests.get(url, headers=headers)
        soup = BeautifulSoup(response.text, "html.parser")
        
        for a in soup.find_all("a", href=True):
            if "/p/" in a['href']:
                full_url = "https://www.flipkart.com" + a['href']
                if full_url not in product_links:
                    product_links.append(full_url)
        
        print(f"  Page {page} → {len(product_links)} total links")
        time.sleep(1)  # polite delay to avoid getting blocked

print(f"\n✅ Total product URLs collected: {len(product_links)}")

Fetching category: laptops
  Page 1 → 29 total links
  Page 2 → 58 total links
  Page 3 → 87 total links
  Page 4 → 116 total links
  Page 5 → 145 total links
Fetching category: mobiles
  Page 1 → 174 total links
  Page 2 → 203 total links
  Page 3 → 232 total links
  Page 4 → 261 total links
  Page 5 → 290 total links
Fetching category: headphones
  Page 1 → 335 total links
  Page 2 → 380 total links
  Page 3 → 425 total links
  Page 4 → 468 total links
  Page 5 → 511 total links

✅ Total product URLs collected: 511


In [5]:
def load_url_data(url:list): 
    loader=UnstructuredURLLoader(urls=product_links)
    documents=loader.load()
    return documents

In [6]:
def text_splitter(data): 
    text_split=RecursiveCharacterTextSplitter(chunk_size=10000,chunk_overlap=100)
    text_chunk=text_split.split_documents(data)
    return text_chunk

In [7]:
extracted_data=load_url_data(url)
text_chunk=text_splitter(extracted_data)

In [8]:
from langchain.embeddings import HuggingFaceEmbeddings

def download_embeddings(): 
    embeddings=HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
    return embeddings

In [9]:
embeddings=download_embeddings()

C:\Users\Mayur kadam\AppData\Local\Temp\ipykernel_17240\1808802536.py:4: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 0.3.0. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFaceEmbeddings`.
  embeddings=HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
c:\Users\Mayur kadam\.conda\envs\chatbot\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [10]:
from langchain.vectorstores import Chroma 

dir="vector_database" 

vector_data=Chroma.from_documents(
    documents=text_chunk,
    persist_directory=dir,
    embedding=embeddings
)

In [11]:
vector_data=Chroma(
    persist_directory=dir,
    embedding_function=embeddings
)

C:\Users\Mayur kadam\AppData\Local\Temp\ipykernel_17240\2802478731.py:1: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 0.4. An updated version of the class exists in the langchain-chroma package and should be used instead. To use it run `pip install -U langchain-chroma` and import as `from langchain_chroma import Chroma`.
  vector_data=Chroma(


In [12]:
retrival=vector_data.as_retriever(search_type="similarity",search_kwagrs={"k":3})

In [22]:
result=retrival.invoke("vivo v60")

In [23]:
print(result)

[Document(metadata={'source': 'https://www.flipkart.com/vivo-t4-5g-phantom-grey-256-gb/p/itmc2634fd4daeb9?pid=MOBHATXZGEAN2FWT&lid=LSTMOBHATXZGEAN2FWTXTUH1P&marketplace=FLIPKART&q=mobiles&store=tyy%2F4io&srno=s_5_109&otracker=search&fm=organic&iid=f3076ba9-3ab9-4120-8f50-50457e27bf96.MOBHATXZGEAN2FWT.SEARCH&ppt=None&ppn=None&ssid=0bj8grr2rk0000001762779986645&qH=eb4af0bf07c16429'}, page_content="Flipkart\n\nExplore Plus\n\n\n\nLogin\n\nBecome a Seller\n\nMore\n\nCart\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nvivo T4 5G (Phantom Grey, 256 GB)\n\nHome\n\nMobiles & Accessories\n\nMobiles\n\nvivo Mobiles\n\nvivo T4 5G (Phantom Grey, 256 GB) (8 GB RAM)\n\nShare\n\nvivo T4 5G (Phantom Grey, 256 GB) (8 GB RAM)\n\n4.5\n\n\n\n80,648 Ratings & 3,676 Reviews\n\n\n\nExtra ₹5000 off\n\n₹22,999\n\n₹27,999\n\n17% off\n\n+ ₹79 Protect Promise Fee Learn more\n\nSecure delivery by 13 Nov, Thursday\n\nAvailable offers\n\n\n\nBank Offer5% cashback on Axis Bank Flipkart

In [ ]:
from langchain.chains.retrieval import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain 
from langchain_core.prompts import ChatPromptTemplate
from langchain_community.llms import Ollama 

llm=Ollama(model="gemma3")

system_prompt=(
    "you are question answered AI assitance "
    "you are friendly chatbot "
    "answer length maximum three sentences "
    "\n\n"
    "{context}"
)

prompt=ChatPromptTemplate.from_messages(
    [
        ("system",system_prompt),
        ("human","{input}"),
    ]
)

In [16]:
question_answer_chain=create_stuff_documents_chain(llm=llm,prompt=prompt)
rag_chain=create_retrieval_chain(retrival,question_answer_chain)

In [19]:
response=rag_chain.invoke({"input":"i am looking for camera phone so which one i should buy iphone 13 or vivo v60"})

In [20]:
print(response["answer"])

Okay, let's break down the iPhone 13 versus the Vivo V60 for a camera phone. This is a classic comparison, and the "best" depends on your priorities.

**iPhone 13 – The Reliable & Versatile Choice**

* **Image Quality (Overall):** The iPhone 13 consistently delivers excellent image quality. Apple’s processing is very good at producing natural-looking photos with accurate colors.
* **Video Recording:** This is where the iPhone 13 *really* shines. It’s widely considered one of the best smartphone cameras for video, offering exceptional stabilization, dynamic range, and quality.
* **Ease of Use:** The iPhone camera app is very intuitive and easy to use, even for beginners.
* **Ecosystem:** If you're already invested in the Apple ecosystem (Mac, iPad, AirPods), the iPhone will integrate seamlessly.
* **Longevity:** Apple generally provides longer software update support than most Android manufacturers, meaning your phone will stay current for longer.
* **Price:** Typically more expensive t